In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.warp import reproject, calculate_default_transform, Resampling
from rasterio.features import shapes
from rasterio.features import rasterize
from rasterio.transform import from_origin
import rasterio.mask
from rasterio.io import MemoryFile
from shapely.geometry import shape
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:

powerdensity_tif_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/GBR_power-density_100m.tif"
uk_boundary_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk boundary.geojson"

uk = gpd.read_file(uk_boundary_path)
uk["geometry"] = uk.geometry.make_valid()
uk = uk[uk.geometry.notna() & ~uk.geometry.is_empty]
geoms = [uk.unary_union]

with rasterio.open(powerdensity_tif_path) as src:
    arr, out_transform = rasterio.mask.mask(src, geoms, crop=True, filled=True, nodata=np.nan)
    pd100 = arr[0].astype("float32")
    if src.nodata is not None:
        pd100 = np.where(pd100 == src.nodata, np.nan, pd100)

h, w = pd100.shape
left, top = out_transform * (0, 0)
right, bottom = out_transform * (w, h)
extent = (left, right, bottom, top)

vmin = float(np.nanmin(pd100))
vmax = 1500

cmap = LinearSegmentedColormap.from_list("blue_yellow", ["blue", "yellow"])
cmap.set_bad(alpha=0)

fig, ax = plt.subplots(figsize=(9, 11), dpi=600)
im = ax.imshow(pd100, extent=extent, origin="upper", cmap=cmap, vmin=vmin, vmax=vmax)

uk.boundary.plot(ax=ax, color="black", linewidth=0.3)

ax.set_axis_off()

cbar = plt.colorbar(im, ax=ax, fraction=0.035, pad=0.02)
cbar.set_label("Power density (W/m²)")

plt.tight_layout()
plt.show()

In [ ]:

threshold = 400.0

uk = gpd.read_file(uk_boundary_path)
uk["geometry"] = uk.geometry.make_valid()
uk = uk[uk.geometry.notna() & ~uk.geometry.is_empty]
geoms = [uk.unary_union]

with rasterio.open(powerdensity_tif_path) as src:
    arr, out_transform = rasterio.mask.mask(src, geoms, crop=True, filled=True, nodata=np.nan)
    pd100 = arr[0].astype("float32")
    if src.nodata is not None:
        pd100 = np.where(pd100 == src.nodata, np.nan, pd100)

h, w = pd100.shape
left, top = out_transform * (0, 0)
right, bottom = out_transform * (w, h)
extent = (left, right, bottom, top)

mask = np.where(np.isfinite(pd100), (pd100 > threshold).astype(np.uint8), np.nan)
mask = np.ma.masked_invalid(mask)

cmap = ListedColormap(["purple", "green"])
cmap.set_bad(alpha=0)

fig, ax = plt.subplots(figsize=(9, 11), dpi=600)
ax.imshow(mask, extent=extent, origin="upper", cmap=cmap, vmin=0, vmax=1)

uk.boundary.plot(ax=ax, color="black", linewidth=0.3)

ax.legend(
    handles=[
        Patch(facecolor="green", edgecolor="none", label=f"> {threshold:.0f} W/m²"),
        Patch(facecolor="purple", edgecolor="none", label=f"≤ {threshold:.0f} W/m²"),
    ],
    loc="upper left",
    frameon=True
)

ax.set_axis_off()
plt.tight_layout()
plt.show()
